# LOCAL CSV to SQL

In [ ]:
import pandas as pd
import pandas.io.sql as sqlio
import pymysql
import psycopg2
from sqlalchemy import create_engine
import os
from os.path import isfile, join

### SET SQL SERVER CONNECTION

In [ ]:
#PostgreSQL
p_host = 'localhost'
p_port = 5432
p_dbname = 'DS_DH'
p_username = ''
p_pwd = ''

#MySQL
m_host = 'localhost'
m_port = 3306
m_dbname = 'DS_DH'
m_username = 'root'
m_pwd = 'root'


p_engine = create_engine('postgresql://'+p_username+':'+p_pwd+'@'+p_host+':'+str(p_port)+'/'+p_dbname)
m_engine = create_engine('mysql+pymysql://'+m_username+':'+m_pwd+'@'+m_host+':'+str(m_port)+'/'+m_dbname, echo=False)

## FUNÇÕES PARA JOGAR O CONTEÚDO DE UM OU MAIS ARQUIVOS PARA MySQL OU POSTGRESQL

In [ ]:
def arquivo(engine,mypath,fname,fid):
    df = pd.read_csv(mypath+fname)
    tablename = fname[:-4]
    print('tablename:', tablename)
    df.to_sql(name=tablename, con=engine, if_exists='replace')
    query = 'SELECT * FROM ' + tablename
    df1 = sqlio.read_sql_query(query, engine)
    if len(df.index) == len(df1.index):
        print(fname, 'subiu com sucesso')
    else:
        print(fname, 'deu ERRO')
    return "OK"

def arquivos(cur,mypath,files):
    for file in files:
        fname = file
        fid = files["file"]
        tablename = fname[:-4]
        print('file:', fname, 'tablename:', tablename)
        df = pd.read_csv(mypath+fname)
        df.to_sql(name=tablename, con=engine, if_exists='replace')
        query = 'SELECT * FROM ' + tablename
        df1 = sqlio.read_sql_query(query, cur)
        if len(df.index) == len(df1.index):
            print(fname, 'subiu com sucesso')
        else:
            print(fname, 'deu ERRO')
    return "OK"


### FORMAS DE LISTAR OS ARQUIVOS DE UMA PASTA

In [ ]:
from os.path import isfile, join
mypath = '/Users/eduardo/OwnCloud/WORK/Microsoft/xbox/DB/Exports/BGS/2017/1010/' #deve terminar em "/" para funcionar na função
files = os.listdir(mypath)
print(files)
files = os.walk(mypath)
print(files)
files = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
print(files)

### CONFIGURAÇÃO DE QUAIS ARQUIVOS DESEJA SUBIR PARA O SQL

Entre com o nome do arquivo que deseja subir para o SQL, ou digite '*' para subir todos os arquivos de uma vez e precione "enter", depois digite "mysql" ou "postgresql" para selecionar em qual banco deseja subir seu arquivo.

In [ ]:
print("Digite o nome do arquivo que deseja subir com a extenção ou '*' para subir todos de uma vez:")
fname = input()

print("Digite 'mysql' ou 'postgresql' para definir em qual banco deseja subir a tabela:")
eng = input()

if eng == 'mysql':
  engine = m_engine
elif eng == 'postgresql':
  engine = p_engine
else:
  print("ERRO: você não digitou a engine do banco corretamente")

if fname == "*":
    subindo = arquivos(engine,mypath,files)
    print(subindo)
else:
    fid = files[fname]
    subindo = arquivo(engine,mypath,fname,fid)
    print(subindo)

In [ ]:
df